In [61]:
# import libs

import numpy as np
import Laser
import gCode
import random


# Important definition
positive angle in CCW direction

### Units
Distances: mm  
Angles: degrees

# Define functions

In [62]:
# read G-Code file and return a list of strings
def read_gcode_file(filename):
    tmp = []
    with open(filename, 'r') as f:
        gcode = f.readlines()
    for line in gcode:
        tmp.append(line)
    
    return np.array(tmp)
    

In [63]:
def parseCoord(gCodeObj,linestring):
    line = linestring
    X = None
    Y = None
    Z = None
    move = False

    # check the command of the gcode line (string until first space)
    command = line.split(' ')[0]

    # check if line is ;LAYER:1 and gCodeObj.firstLayerCheck is False
    if line.startswith(';LAYER:1') and not gCodeObj.firstLayerCheck:
        gCodeObj.updateFirstLayerCheck(True)
        print("First layer check is True")

    # check if firstLayerCheck is True
    if gCodeObj.firstLayerCheck:
        # if command == G1 or G0 than move = True
        if command == 'G1' or command == 'G0':
            move = True
        
        # check if line contains ; which means it is a comment and skip line
        if not ';' in line:
            # parse the X Y Z coords from string to float
            if 'X' in line:
                X = float(line.split('X')[-1].split(' ')[0])
            elif 'X' not in line:
                X = None
            if 'Y' in line:
                Y = float(line.split('Y')[-1].split(' ')[0])
            elif 'Y' not in line:
                Y = None
            if 'Z' in line:
                Z = float(line.split('Z')[-1].split(' ')[0])
            elif 'Z' not in line:
                Z = None
    else:
        X = None
        Y = None
        Z = None
    return move, X, Y, Z, command, linestring


In [64]:
# calculate the distance between two points
def calcDistance(x0, y0, x1, y1):
    return np.sqrt((x1-x0)**2 + (y1-y0)**2)


In [65]:
# calculate angle between two points (atan2)
def calcAngle(x0, y0, x1, y1):
    newAngle = np.arctan2((y1-y0),(x1-x0))
    if newAngle < 0:
        newAngle = 2*np.pi - np.abs(newAngle)
    # convert to degrees
    newAngle = newAngle*180/np.pi
    return newAngle

In [66]:
def findClosestAngle(targetAngle, LaserObj):
    currentAngles = LaserObj.getNormAngles()
    # find the closest angle in the list of angles
    closestAngleIndx = np.argmin(np.abs(currentAngles - targetAngle))
    closestAngle = currentAngles[closestAngleIndx]

    # calculte delta bewteen target and closest angle
    deltaAngle = targetAngle - closestAngle

    # update the angle of the laser
    LaserObj.updateAngles(deltaAngle)

    return closestAngleIndx, deltaAngle

    

In [67]:
def convertAngleToSteps(angle):
    # function to convert an angle to steps for the stepper motor
    ratioGear = 37/24
    ratioStepper = 0.9

    steps = angle/ratioGear
    steps = steps/ratioStepper

    #return steps // unit of angles and conversion is made in the config file of the printer
    return angle

In [68]:
def postProcessGcode(gCodeArray, threshold, LaserObj, newFilePath, gCodeObj):

    for i in range(gCodeArray.shape[0]-1):
        move0, X0, Y0, Z0, command0, line0 = parseCoord(gCodeObj, gCodeArray[i])
        move1, X1, Y1, Z1, command1, line1 = parseCoord(gCodeObj,gCodeArray[i+1])
       
        # check if both lines are moves
        #print("move0", move0, "move1", move1)
        if move0 and move1:
            # check if both lines have no Z
            if Z0 == None and Z1 == None:
                # check if the distance between the two points is larger than the threshold
            
                #print("i:", i+1, "move0: ", move0, "move1: ", move1, "X0: ", X0, "Y0: ", Y0, "Z0: ", Z0, "command0: ", command0, "line0: ", line0, "X1: ", X1, "Y1: ", Y1, "Z1: ", Z1, "command1: ", command1, "line1: ", line1)
                dist = calcDistance(X0, Y0, X1, Y1)
                if dist > threshold:

                    # calculate the angle between the two points
                    targetAngle = calcAngle(X0, Y0, X1, Y1)
                    # find the closest angle to the target angle
                    closestAngleIndx, deltaAngle = findClosestAngle(targetAngle, LaserObj)
                    # update the angle of the laser
                    LaserObj.updateAngles(deltaAngle)
                    # convert the angle to steps
                    steps = convertAngleToSteps(deltaAngle)
                    # add new gcode line
                        # rotate the laser by delta angle
                    newLine01 = "A" + str(steps) + "; rotate laser by " + str(deltaAngle) + " degrees"
                        # activate laser PWM signal
                    newLine02 = "M index:" + str(closestAngleIndx)+ "=100 ;activate laser"
                        # insert line0
                    newLine03 = line0
                        # deactivate laser PWM signal
                    newLine04 = "M index:" + str(closestAngleIndx)+ "=0 ;deactivate laser" + '\n'

                    lines = [newLine01, newLine02, newLine03, newLine04]

                    # append new lines to newFilePath
                    with open(newFilePath, 'a') as f:
                        f.write(newLine01 + '\n')
                        f.write(newLine02 + '\n')
                        f.write(newLine03 + '\n')
                        f.write(newLine04)
                    f.close()
        else:
            # append line0 to newFilePath
            with open(newFilePath, 'a') as f:
                f.write(line0)
            f.close()

    return True

# Test functions

In [69]:
# test calcAngle function
if (0):
    print(calcAngle(0,0,0,0))
    print(calcAngle(0,0,1,-1))
    print(calcAngle(0,0,-1,-1))
    print(calcAngle(0,0,-1,1))


In [70]:
# test the parseCoord function
if(0):

# read the gcode file
    gcode = read_gcode_file("GCODE.txt")

    # create empty lists for the coordinates
    X = []
    Y = []
    Z = []
    cmd = []
    i = 0
    # loop over all lines in the gcode file
    for line in gcode:
        #print(i)
        #i += 1
        # parse the coordinates from the line
        moveBool, x, y, z, command, linestring = parseCoord(line)
        # append the coordinates to the lists
        X.append(x)
        Y.append(y)
        Z.append(z)
        cmd.append(command)

# print first 10 elements of X Y Z
j = 50
if(0):
    for i in range(j):
        print(f"X: {X[i]} Y: {Y[i]} Z: {Z[i]} cmd: {cmd[i]}")

# Main 

In [71]:
# read the gcode file
gcode = read_gcode_file("GCODE.txt")
gCodeObj = gCode.gCode("gCode")

# print shape of gcode
print(gcode.shape)

# create laser object
numberOfLaser = 4
laser = Laser.Laser(numberOfLaser)

# threshold for the distance between two points 
minDistance = 2

# create new file
newFilePath = "GCODE_postprocessed.txt"

# post process the gcode
print(postProcessGcode(gcode, minDistance, laser, newFilePath, gCodeObj))


(10045,)
First layer check is True
True


In [73]:
print(gCodeObj.firstLayerCheck)
print(laser.absLaserMovement)

True
347331.0781893332
